In [7]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split

data_path = Path("./winequality-white.csv")
data_columns = ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", 
"pH", "sulphates", "alcohol", "quality"]
wine_df = pd.read_csv(data_path, header=0, names=data_columns, sep=";")

test_size=0.2
train_df, test_df = train_test_split(wine_df, test_size=test_size)

# data overview:
#print(wine_df.describe())

# which quality classes do we have? :
qualities = wine_df["quality"].unique()
print(f"Number of unique 'qualities': {len(qualities)}")
print(f"Qualities: {sorted(qualities)}")
binc = np.bincount([q for q in wine_df["quality"]])
no_inst = len(wine_df)
print(f"\nClass counts: {binc}")
print(f"\nNumber of instances: {no_inst} ")
print(f"\nClass fractions: {np.round(binc/no_inst,4) * 100}")


Number of unique 'qualities': 7
Qualities: [3, 4, 5, 6, 7, 8, 9]

Class counts: [   0    0    0   20  163 1457 2198  880  175    5]

Number of instances: 4898 

Class fractions: [ 0.    0.    0.    0.41  3.33 29.75 44.88 17.97  3.57  0.1 ]


In [181]:
from sklearn.preprocessing import StandardScaler

def scale_dataframe(data_df, exempt_last_column=True,column_names_to_scale=None):
    """
        Scales columns of a given data frame with a StandardScaler from Sklearn. 
        Input:
            data_df : dataframe with numerical values to normalize
            exempt_last_column : if true, column_names_to_scale will be ignored and all but the last column will be scaled.
            column_names_to_scale : list of the names of the columns to be scaled

        Output:
            dataframe with columns scaled
    """
    scaler = StandardScaler()
    if(exempt_last_column):
        data = data_df.to_numpy()
        data_to_scale = data[:,:-1]
        last_column = np.expand_dims(data[:,-1].astype(np.int_), axis=1)
        data_scaled = np.append(scaler.fit_transform(data_to_scale), last_column, axis=1)
        return pd.DataFrame(data_scaled)
    elif(column_names_to_normalize):
        data_to_scale = data_df[column_names_to_scale].to_numpy()
        data_scaled = scaler.fit_transform(data_to_scale)
        df_temp = pd.DataFrame(data_scaled, columns=column_names_to_scale, index=data_df.index)
        data_df[column_names_to_scale]= df_temp
    else:
        data_to_scale = data_df.to_numpy()
        data_scaled = scaler.fit_transform(data_to_scale)
        data_df = pd.DataFrame(data_scaled)
    
    return data_df


In [12]:
# Model definition from the WineDataset note book -- I don't know how to import from another Jupyter notebook...
class WineNetwork(nn.Module):
    def __init__(self):
        super(WineNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(11, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(p=0.2),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(256, 10),
            nn.ReLU()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [194]:
def train_loop(dataloader, model, loss_fn, optimizer):
    losses, nof_correct = 0, 0
    for xx, y_true in dataloader:
        y_pred = model(xx)
        loss= loss_fn(y_pred, y_true)
        losses += loss.item()
        nof_correct += (y_pred.argmax(1) == y_true).sum().item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return losses, nof_correct

def test_loop(dataloader, model, loss_fn):
    losses, no_correct = 0, 0
    with torch.no_grad():
        for (X,y_true) in dataloader:
            pred = model(X)
            losses += loss_fn(pred, y_true).item()
            no_correct += (pred.argmax(1)== y_true).sum().item()
     
    return losses, no_correct
    

In [186]:
# transform the training data with the decision-tree:

import pickle

Wine_tree_filename = 'AdaBoost_071_model.dct'

def Wine_tree_map(data_df, tree_name):
    """
    Maps a data-frame by using a pre-trained decision-tree.
    Input:
        data_df : Pandas data-frame to be mapped
        tree_name : path / name of the pretrained tree
    Returns:
        Pandas data-frame
    """
    tree_model = pickle.load(open(tree_name, 'rb'))
    X = data_df.to_numpy()
    # "np.int_" is "long" in numpy:
    # predict_proba:
    #Z = np.append(tree_model.predict_proba(X[:,:-1]), np.expand_dims(X[:,-1].astype(np.int_), axis=1), axis=1)
    
    # predict:
    Z = np.append(np.expand_dims(tree_model.predict(X[:,:-1]).astype(np.int_), axis=1), np.expand_dims(X[:,-1].astype(np.int_), axis=1), axis=1)
    
    return pd.DataFrame(Z)

In [164]:
# maps the training data with the neural-net:

import numpy as np
import pandas as pd


WineNetwork_filename = "model_640_0.001_369_64_SGD.pt"

def WineNetwork_map(data_df, net_name):
    """
    Maps a dataframe by applying a pre-trained neural net of type "class WineNetwork"
    Input:
        data_df : the pandas data-frame to be transformed
        net_name : the path / name of the neural net of type WineNetwork to be loaded
    Returns:
        a pandas data-frame        
    """
    class WineNetwork(nn.Module):
        def __init__(self):
            super(WineNetwork, self).__init__()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(11, 64),
                nn.ReLU(),
                nn.BatchNorm1d(64),
                nn.Dropout(p=0.2),
                nn.Linear(64, 128),
                nn.ReLU(),
                nn.BatchNorm1d(128),
                nn.Linear(128, 256),
                nn.ReLU(),
                nn.BatchNorm1d(256),
                nn.Linear(256, 10),
                nn.ReLU()
            )

        def forward(self, x):
            logits = self.linear_relu_stack(x)
            return logits

    net_model = WineNetwork()
    net_model.load_state_dict(torch.load(net_name))
    net_model.eval()
    # softm = lambda x : np.exp(x)/np.sum(np.exp(x))
    softm = torch.nn.Softmax(dim=1)
    
    X = torch.tensor(data_df.iloc[:,:-1].to_numpy(), dtype=torch.float32).detach()
    Y = torch.tensor(data_df.iloc[:, -1].to_numpy(), dtype=torch.long).detach()

    Z = torch.cat((softm(net_model(X)), Y.unsqueeze(dim=1)), dim=1).detach().numpy()
    
    return pd.DataFrame(Z)
    

In [165]:
# define torch.dataset: __init__(), __len__(), __getitem__()
from torch.utils.data import Dataset

class WineDataSet(Dataset):
    def __init__(self, data_df, transform=None, target_transform=None):
        self.data_df = data_df
        self.transform = transform
        self.target_transform = target_transform
        self.X = torch.tensor(self.data_df.iloc[:,:-1].to_numpy(), dtype=torch.float32)
        self.Y = torch.tensor(self.data_df.iloc[:, -1].to_numpy(), dtype= torch.long)

    def __len__(self):
        return len(self.Y)
        
    def __getitem__(self,idx):
        self.x = self.X[idx,:]
        self.y = self.Y[idx]
        if self.transform != None:
            self.x = self.transform(self.x)
        if self.target_transform != None:
            self.y = self.target_transform(self.y)
        return self.x, self.y

In [191]:
# load data:
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

data_path = Path("./winequality-white.csv")
column_names = ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", 
"pH", "sulphates", "alcohol", "quality"]
#column_names_to_normalize = column_names[:-1]
wine_df = pd.read_csv(data_path, header=0, names=column_names, sep=";")

# prepare data:
shuffle(wine_df, random_state=0)
scaled_wine_df = normalize_dataframe(wine_df, exempt_last_column=True)

WineNetwork_filename = "model_640_0.001_369_64_SGD.pt"
Wine_tree_filename = 'AdaBoost_071_model.dct'

net_df = WineNetwork_map(scaled_wine_df, net_name=WineNetwork_filename)
net_df = normalize_dataframe(net_df)
tree_df = Wine_tree_map(wine_df, tree_name=Wine_tree_filename)
tree_df = normalize_dataframe(tree_df)

# combine net_df and tree_df to combined_data_df:
combined_data_df = pd.DataFrame(np.append(net_df.iloc[:,:-1].to_numpy(), tree_df.to_numpy(), axis=1))

# split into train_df, test_df:
test_size = 0.2
combined_train_df, combined_test_df = train_test_split(combined_data_df, test_size=test_size)
blender_input_dim = combined_data_df.shape[1]-1

# create dataloader from train_df and test_df:
batch_size=64
train_ds = WineDataSet(combined_train_df)
test_ds = WineDataSet(combined_test_df)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

In [192]:
blender_input_dim

11

In [179]:
import torch
from torch import nn

class BlenderModel(nn.Module):
    
    def __init__(self, input_dim):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
        nn.Linear(input_dim, 128),
        nn.ReLU(),
        nn.BatchNorm1d(128),
        nn.Dropout(p=0.2),
        nn.Linear(128, 10),
        nn.ReLU(),
        )
    
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits


In [196]:
# Train the model:
import os

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter


# writer for tensorboard:
writer = SummaryWriter()

# parameters:

# loss function:
# cross-entropy:
loss_fn = nn.CrossEntropyLoss()

# optimizer:

# adam:
optimizer_name = "ADAM"
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# sgd:
#optimizer_name = "SGD"
#learning_rate = 1e-3
#optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate, momentum=0.9)

epochs = 100
write_log_after_epochs = 20
best_model_name = ""
max_correct = -torch.inf

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model = BlenderModel(blender_input_dim)

for ep in range(1, epochs+1):        
       
        # put model in train mode:
        model.train()
        (train_loss, train_no_correct) = train_loop(train_dl, model, loss_fn, optimizer)
              
        # switch model to to evaluation mode:
        model.eval()
        (test_loss, test_no_correct) = test_loop(test_dl, model, loss_fn)

        if(test_no_correct > max_correct):
            max_correct = test_no_correct
            if(best_model_name):
                os.remove(best_model_name)
            best_model_name = "./blender_model_" + str(test_no_correct) + "_" + str(learning_rate) + "_" + str(ep) + "_" + str(batch_size) + "_" + optimizer_name + "_sv.pt"
            torch.save(model.state_dict(), best_model_name)

        writer.add_scalar("Loss/test", test_loss/ len(test_ds), ep)
        writer.add_scalar("Accuracy/test", test_no_correct/ len(test_ds), ep)
        writer.add_scalar("Loss/train", train_loss/ len(train_ds), global_step=ep)
        writer.add_scalar("Accuracy/train", train_no_correct/ len(train_ds), global_step=ep)
       
        if ep % write_log_after_epochs == 0:
            print(f"\n----- Epoch: {ep} -----")
            print(f"Epoch loss: {test_loss/ len(test_ds)}")
            print(f"Epoch accuracy: {test_no_correct/ len(test_ds)}")
            

Using device: cpu

----- Epoch: 50 -----
Epoch loss: 0.03956298901110279
Epoch accuracy: 0.0163265306122449

----- Epoch: 100 -----
Epoch loss: 0.03973532501532107
Epoch accuracy: 0.015306122448979591

----- Epoch: 150 -----
Epoch loss: 0.03985172242534404
Epoch accuracy: 0.015306122448979591

----- Epoch: 200 -----
Epoch loss: 0.03962579138424932
Epoch accuracy: 0.0163265306122449


In [119]:
model

BlenderModel(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=17, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=10, bias=True)
    (3): ReLU()
  )
)